# Algebra linear computacional

In [68]:
import numpy as np

## Sistemas lineares

### Substituição para frente(Forward Substitution)
Substituição para Frente é um método utilizado para resolver sistemas de equações lineares na forma:
 $$L y = b$$

Onde : 
* L é uma matriz triangular inferior
* y é o vetor de incógnitas intermediárias
* b é o vetor constantes

Exemplo hipotetico abaixo:

In [69]:
# L
np.tri(N=3, M=5)

array([[1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0.]])

In [107]:
np.random.RandomState(1).randn(5,1).T

array([[ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763]])

#### Teorema 1.3.1

Seja $G$ uma matrix triangular. Então ,$G$ é invertivel se, e somente se, todos elementos da diagonal principal de $G$ são diferentes de zero.

$DET(G) \not = 0$ , se G é invertivel
* $det(G) = g_{11}g_{22}...g_{nn} \not = 0$ para $i = 1, 2, ..., n$

In [72]:
def prova(G):
    try: 
        inv = np.linalg.inv(G)
        return inv
    except Exception:
        raise ValueError("Não é invertivel")

In [73]:
try:
    prova(
        np.array([[1, 2, 2],
              [0, 0, 3],
              [0, 0, 1]])
    )
except Exception:
    print("Não tem inversa")

Não tem inversa


In [74]:
prova(
    np.array([[1,1,1],[0,1,1],[0,0,1]])
)

array([[ 1., -1.,  0.],
       [ 0.,  1., -1.],
       [ 0.,  0.,  1.]])

&nbsp;

#### Transformando a matriz em $L$

In [75]:
def transformL(A):
    # regra de cramer
    if np.linalg.det(A) == 0:
        raise Exception("Não tem inversa")
    
    for i in reversed(range(A.shape[0])):
        u_ = A[i, i]
        
        if i == 0 or u_ == 0: break
        for j in reversed(range(i)):
            L = A[j, i]/u_
            A[j, : ] = np.around(A[j, :] - L * A[i, :], decimals=4)
               
    return A

In [76]:
A = np.array([
    [5, 3, 3],
    [4, 5, 6],
    [2, 4, 7]
])
b = np.around(
    np.random.RandomState(1).randn(3, 1)
    )

In [77]:
A, b

(array([[5, 3, 3],
        [4, 5, 6],
        [2, 4, 7]]),
 array([[ 2.],
        [-1.],
        [-1.]]))

In [78]:
L = transformL(A)

#### Algoritmo de substituição

```pseudo
_______________________________________
ALG.....: ALGORITMO DE SUBSTITUIÇÃO
ENTRADA.: L e b
SAIDA...: y
---------------------------------------
INICIO:
    CRIA Y = [0....][TAMANHO = N]

    Y[1] = b1 / L[1, 1]
    PARA i = 2 FAÇA ATÉ N:
        S = 0 
        PARA j FAÇA ATÉ i-1:
            S += L[i, j] * y[j]

        Y[i] = 1/L[i, i] * (b[i] - S)
    RETURN Y
```

In [79]:
def forwardSubstitution(L, b):
    # criando um vetor y
    y  = np.zeros(shape=(L.shape[1], 1)) 

    # definindo y[0] 
    y[0, :] = b[0, :]/ L[0, 0]

    # continuando o alg de substituição
    for i in range(1, L.shape[0]):
        S = np.sum([ L[i,j] * y[j] for j in range(i) ])
        
        y[i, :] = 1/L[i,i] * (b[i, :] - S )
    return y

In [80]:
y = forwardSubstitution(L, b)
y

array([[ 1.        ],
       [-3.        ],
       [ 1.28571429]])

In [81]:
np.linalg.solve(A, b)

array([[ 1.        ],
       [-3.        ],
       [ 1.28571429]])

Outro exemplo

In [82]:
A = np.random.RandomState(1).randn(5, 5)*10
A = np.around(A, decimals=2)

b = np.random.RandomState(1).randn(5, 1)*1
b = np.around(b, decimals=2)

A, b

(array([[ 16.24,  -6.12,  -5.28, -10.73,   8.65],
        [-23.02,  17.45,  -7.61,   3.19,  -2.49],
        [ 14.62, -20.6 ,  -3.22,  -3.84,  11.34],
        [-11.  ,  -1.72,  -8.78,   0.42,   5.83],
        [-11.01,  11.45,   9.02,   5.02,   9.01]]),
 array([[ 1.62],
        [-0.61],
        [-0.53],
        [-1.07],
        [ 0.87]]))

In [83]:
L = transformL(A) 

In [84]:
forwardSubstitution(L, b)

array([[-0.06578279],
       [-0.14954343],
       [ 0.0508016 ],
       [ 0.68862814],
       [-0.22831741]])

In [85]:
np.linalg.solve(A, b)

array([[-0.06578279],
       [-0.14954343],
       [ 0.0508016 ],
       [ 0.68862814],
       [-0.22831741]])

&nbsp;

## Decomposição de $\text{Choesky}$

É a decomposição de uma matriz simetrica e definida positiva no produto de uma matriz triangular inferior e sua transpota($A=LL^t$)
Quando aplicável, a decomposição é aproximadamente duas vezes mais eficiente que a decomposição LU para resolver sistemas de equações linares.

#### Condições

**Simetrica:** $A$ deve ser simétrica ($A=A^t$).

**Positividades:**
1. Todos os menores principais dominantes dever ser positivos
2. Todos os autovalores devem ser positivos

exemplo:

In [86]:
# Verificando se a matrix é simetrica
def isSimetric(A):
    return np.all(A == A.T)

In [87]:
A_ = np.array([
    [2, 1, 1],
    [1, 3, 1],
    [1, 1, 2]
])

In [88]:
isSimetric(A_)

np.True_

In [89]:
# Verificando a positividade da matriz
def Positividade(A):
    return np.all(
        np.linalg.eigvals(A) >0
    )
    

In [90]:
Positividade(A_)

np.True_

&nbsp;

#### Algoritmo de $\text{cholesky}$

$$
\text{Inicialize L com uma matriz triangular inferior com seros} \\

1.L_{j,j} = \sqrt{A_{j,j} - \sum_{k-1}^{j-1}{L_{j,k}}^2} \\
2. L_{i,j} = \frac{1}{L_{i,j}} ({A_{j,j} - \sum_{k-1}^{j-1}{L_{j,k}} \cdot L_{i,k}}) , \text{para } i> j
$$


In [104]:
def cholesky(A):
    N, M = A.shape
    L = np.tri(N, M)

    for i in range(A.shape[0]):
        for j in range(i):
            L[i, j] = (1/L[j, j]) * (A[i, j] - np.sum([L[j, k] * L[i, k] for k in range(j)]))
       
        L[i, i] = np.sqrt(A[i, i] - np.sum([L[i, k]**2 for k in range(i)]))
    return L       

In [105]:
L = cholesky(A_)

[[1. 0. 0.]
 [1. 1. 0.]
 [1. 1. 1.]]
3 3


In [106]:
L@L.T, L

(array([[2., 1., 1.],
        [1., 3., 1.],
        [1., 1., 2.]]),
 array([[1.41421356, 0.        , 0.        ],
        [0.70710678, 1.58113883, 0.        ],
        [0.70710678, 0.31622777, 1.18321596]]))

&nbsp;

## Decomposição $LU$

É a fatoração de uma matriz quadrada A em duas matrizes: 
* $L$ triangular inferior
```pseudo
[1, 0, 0]
[1, 1, 0]
[1, 1, 1]
```
* $U$ triangular superior
```pseudo
[1, 1, 1]
[0, 1, 1]
[0, 0, 1]
```

A decomposiçã $LU$ facilita a resoluçã de sistemas lineares, especialmente quando o sistema é a parte do múltiplos sistemas com a mesma matriz de coeficientes. Foi desenvolvidas a partir do método de eleminação de Gauss, adaptando-o para uma fatoração matricial

&nbsp;

#### Vantagem Computacional

| Tradicional | LU |
| ----------- | -- |
| $O(n^3)$ opereções para cada sistema   | Fatorar $A = LU$ uma única vez: $O(n^3)$ operações|
|         | Para cada novo sistema, resolver $Ly = b$, Ux=y. "Apenas" $O(n^2)$ operações por sistema|

#### Condições
Existencias:
A matriz deve ser nao singular. Eventualmente, pode ser necessário alterar a ordem das linhas e/ou colunas de $A$ para evitar pivos nulos(**pivoteamento**)
$$
PA=LU
$$
Onde $P$ é matriz de permutação

>uma matriz é chamada de singular se:
> 1. Seu determinante é igual a zero (det(𝐴)=0)
> 2. Ela não possui inversa.
> 

In [108]:
def isSingular(A):
    try:
        if np.isclose(np.linalg.det(A), 0):
            print("det = 0")
        else:
            print("det dif 0")
        
        np.linalg.inv(A)
        print("Possui inversa")
    except np.linalg.LinAlgError:
        print("Não possui inversa")

In [109]:
isSingular(A)

det dif 0
Possui inversa
